# hVAE

In [2]:
from src.hvae import hVAE

PATH = "path"

hvae = hVAE.load(PATH)
hvae.eval()
hvae.generate(10)

TypeError: load() missing 1 required positional argument: 'path'

In [ ]:
from torch import nn

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)


vae = VAE()
for p in vae.parameters():
    print(p.shape)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def visualize_graph():
    G = nx.DiGraph()

    encoder_edges = [('x', 'z1'), ('y', 'z1'), ('x', 'y')]
    decoder_edges = [('z1', 'x'), ('y', 'z1')]

    # use the same node list for both nx.draw calls
    nodes = ['x', 'y', 'z1']

    # specify the positions of the nodes
    pos = {'x': (0, 0), 'z1': (0, 1), 'y': (0, 2)}

    nx.draw(G, nodelist=nodes, pos=pos, edgelist=encoder_edges, edge_color="r", width=2, node_size=2000, with_labels=True, node_color="lightblue", ax=plt.gca(), arrowstyle="->", connectionstyle="arc3,rad=0.2")

    # draw the blue edges with labels and arrows on the same axis with negative curvature
    nx.draw(G, nodelist=nodes, pos=pos, edgelist=decoder_edges, edge_color="b", width=2, node_size=2000, with_labels=True, node_color="lightblue", ax=plt.gca(), arrowstyle="->")
    # show the plot
    plt.show()


In [ ]:
visualize_graph()

In [ ]:
from hparams import get_hparams

hparams = get_hparams()
model = hparams.model_params.model()

In [ ]:
for param in model.parameters():
    print(param.size())

In [ ]:
from torchinfo import summary
summary(model, (1, 784), depth=7)

In [ ]:
from test import main
main()

In [1]:
from train import main
main()

2023-08-09 20:07:51,540 - INFO - Train step generator trainable params 18.361m.
2023-08-09 20:07:51,595 - INFO - (0, ('Time/Step (sec)', 0.05), ('Reconstruction Loss', 18.631), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 169745312.0), ('GradSkipCount', 1.0))
2023-08-09 20:07:51,595 - INFO - 
-----------------------------------------------------------------------------------------
2023-08-09 20:07:51,601 - INFO - Train Stats for global_step 0 | NELBO nan | SSIM: 0.14244940876960754


tensor([[   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan, 3.7052,    nan],
        ...,
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan]],
       grad_fn=<SubBackward0>)
tensor([[4.1934e+00,        nan,        nan,  ...,        nan, 1.2028e-02,
                nan],
        [1.6759e+00, 8.0438e-01,        nan,  ..., 3.4160e-01,        nan,
         5.7346e-01],
        [       nan, 6.1879e-01, 4.5482e+00,  ..., 7.3859e-01,        nan,
         4.1270e-01],
        ...,
        [1.6030e+01, 4.4055e-01, 1.0477e+03,  ..., 2.3607e-01, 2.9318e-03,
                nan],
        [       nan,        nan,        nan,  ...,        nan, 1.8685e+00,
         1.0120e+00],
        [5.9215e-01,        nan, 2.1920e+00,  ...,        nan, 1.4574e+00

2023-08-09 20:07:54,117 - INFO - Validation Stats for global_step 0 | Reconstruction Loss 37.3869 | KL Div nan |NELBO nan | SSIM: 0.144395
2023-08-09 20:07:54,118 - INFO - Logging to Tensorboard..


tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
tensor([[       nan,        nan,        nan,  ...,        nan,        nan,
                nan],
        [       nan,        nan, 2.6860e+02,  ...,        nan, 3.3578e-01,
         1.3948e+00],
        [1.6622e+00, 4.6944e+00, 2.3960e+01,  ..., 6.1466e-01,        nan,
         4.2632e-02],
        ...,
        [2.8473e+00, 5.9980e-01,        nan,  ...,        nan,        nan,
                nan],
        [       nan, 8.0733e-01,        nan,  ..., 9.0292e-03,        nan,
                nan],
        [1.4655e+00, 1.3052e-01,        nan,  ...,        nan, 7.8762e+00,
         2.0619e+01]])
tensor([[   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    

2023-08-09 20:07:57,322 - INFO - Saved checkpoint for global_step 0 to experiments/TDVAE/2023-08-09_20-07-51/checkpoints/checkpoint-0.pth
2023-08-09 20:07:57,325 - INFO - 
-----------------------------------------------------------------------------------------

2023-08-09 20:07:57,380 - INFO - (1, ('Time/Step (sec)', 0.05), ('Reconstruction Loss', 18.669), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 24790.0), ('GradSkipCount', 2.0))
2023-08-09 20:07:57,426 - INFO - (2, ('Time/Step (sec)', 0.04), ('Reconstruction Loss', 18.713), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 1297714.4), ('GradSkipCount', 3.0))
2023-08-09 20:07:57,469 - INFO - (3, ('Time/Step (sec)', 0.04), ('Reconstruction Loss', 18.613), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 1908417.0), ('GradSkipCount', 4

...

2023-08-09 20:08:02,816 - INFO - Saved checkpoint for global_step 5 to experiments/TDVAE/2023-08-09_20-07-51/checkpoints/checkpoint-5.pth
2023-08-09 20:08:02,817 - INFO - 
-----------------------------------------------------------------------------------------

2023-08-09 20:08:02,862 - INFO - (6, ('Time/Step (sec)', 0.04), ('Reconstruction Loss', 18.61), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 1104778.2), ('GradSkipCount', 7.0))
2023-08-09 20:08:02,905 - INFO - (7, ('Time/Step (sec)', 0.04), ('Reconstruction Loss', 18.624), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 590442.2), ('GradSkipCount', 8.0))
2023-08-09 20:08:02,949 - INFO - (8, ('Time/Step (sec)', 0.04), ('Reconstruction Loss', 18.744), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 4684742.5), ('GradSkipCount', 9

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]],
       grad_fn=<DifferentiableGraphBackward>)
tensor([[       nan,        nan, 8.0924e-01,  ..., 6.9448e-01,        nan,
                nan],
        [5.6543e-01, 4.9754e+03,        nan,  ...,        nan, 9.1908e-01,
         5.0288e-01],
        [       nan,        nan, 3.1556e+02,  ..., 6.4970e-01, 1.2436e+02,
         7.1245e-01],
        ...,
        [       nan, 2.6646e+00, 3.0602e-01,  ..., 7.5946e-01, 1.2618e-01,
         6.7418e-01],
        [2.1210e+01, 1.3156e+00,        nan,  ...,        nan, 1.0126e+00,
         8.4957e+00],
        [2.9872e+00,        nan, 6.3976e+01,  ..., 8.1996e-01,        nan,
                nan]], grad_fn=<DifferentiableGraphBackward>)
tensor([[   nan,    nan,    n

2023-08-09 20:08:03,038 - INFO - (10, ('Time/Step (sec)', 0.04), ('Reconstruction Loss', 18.734), ('KL loss', nan), ('ELBO', nan), ('average KL loss', nan), ('Beta', 0.0001), ('N° active groups', 0), ('GradNorm', 143690.5), ('GradSkipCount', 11.0))
2023-08-09 20:08:03,039 - INFO - 
-----------------------------------------------------------------------------------------
2023-08-09 20:08:03,045 - INFO - Train Stats for global_step 10 | NELBO nan | SSIM: 0.14380866289138794


tensor([[   nan,    nan, 0.7697,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        ...,
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan]])
tensor([[       nan,        nan,        nan,  ...,        nan, 1.2356e+00,
                nan],
        [2.0463e+00,        nan, 4.0359e-01,  ..., 1.7410e+00, 6.6103e-01,
                nan],
        [2.6857e+01, 1.2082e+02,        nan,  ...,        nan, 1.5030e+01,
         5.1904e+00],
        ...,
        [       nan,        nan, 1.0469e+03,  ..., 2.2542e+00, 1.0692e+00,
         9.8391e-01],
        [       nan, 6.1831e-01, 7.2799e+01,  ...,        nan, 1.4155e+00,
         3.7593e+00],
        [5.3304e+00,        nan, 7.1423e-01,  ...,        nan, 4.8149e+01,
         2.4348e+00]])
tensor

KeyboardInterrupt: 